In [10]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind

### Getting Test Playlist From Spotify

In [2]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [3]:
print(token)

BQDIU5idfyG85qHyv17Lhf2m2sq8JJmSAPlwGPotnM-tciVjeoz1WeYgoV56ikMNyqv99YQwYty38N7UCKKi2TcdaHsyE23xrfqT-eAD1IXEYEZBaTvtIhmSjzXZMZ3rHaGVWONF2GoAhR9TJYoTzFXgNWY


In [4]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

#### Vodafone FM 

https://open.spotify.com/playlist/5QZyi299jBblzaTFW1nP4h?si=PawXR_5mSL-ZfezTCUX2Hg

In [5]:
#I'm going to test with a playlist from Vodafone FM 

playlist_index = 0

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri

#just to confirm: spotify:playlist:3M2gjOU19KrxZOCYtVYUCY

'spotify:user:Test_1:playlist:3M2gjOU19KrxZOCYtVYUCY'

In [6]:
uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

In [7]:
results = {'items':[]}

for n in range(0,3000,100):
    new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
    results['items'] += new['items']
    
    playlist_tracks_data = results
    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    
    for track in playlist_tracks_data['items']:
        playlist_tracks_id.append(track['track']['id'])
        playlist_tracks_titles.append(track['track']['name'])
        
    #adds a list of all artists involved in the song to the list of artists for the playlist
        for artist in track['track']['artists']:
            artist_list = []
            artist_list.append(artist['name'])
        playlist_tracks_artists.append(artist_list[0])
        
    
print(len(playlist_tracks_id))

test_df = pd.DataFrame([])
for i in range(0, len(playlist_tracks_id)):
    features = sp.audio_features(playlist_tracks_id[i])
    features_df = pd.DataFrame(features)
    test_df = test_df.append(features_df)

1190


In [8]:
#Merging Dataframes for getting audio features and track information

test_df['title'] = playlist_tracks_titles
#features_df['first_artist'] = playlist_tracks_first_artists
test_df['main_artist'] = playlist_tracks_artists
#features_df = features_df.set_index('id')
test_df = test_df[['id', 'title', 'main_artist',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]

test_df

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.625000,0.03550,0.1160,0.502,143.802,216004,4
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.044300,0.00826,0.1420,0.369,125.389,177520,4
0,6hlJIYn5IBNqcSGTS6X5Xq,My Girls,Animal Collective,0.508,0.953,4,-6.031,0,0.256000,0.04450,0.9290,0.555,119.973,340787,4
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.057700,0.01970,0.3430,0.537,117.783,315200,4
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.111000,0.04490,0.1050,0.301,139.389,335333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,30UWPcZN1mP0P6ZNy7euw1,Putos Bons,Linda Martini,0.177,0.932,0,-5.951,1,0.000007,0.69600,0.3450,0.166,185.187,289007,3
0,7zqhUgWpWzwYnT9eewPJEK,The End,Daughter,0.463,0.688,9,-8.127,1,0.873000,0.00456,0.0666,0.200,128.986,329647,4
0,6ajamBF0Lrmizka3L7dkkE,Marigold,Nice Weather for Ducks,0.304,0.681,0,-6.739,1,0.102000,0.00000,0.0853,0.625,146.049,203053,5
0,013jXbIwEakb8BWbK0bfjO,Someone to Lose,Wilco,0.808,0.572,0,-7.969,1,0.377000,0.29800,0.0927,0.730,122.209,200355,4


#### Test Random Forest

In order to test my best classifier (Random Forest):

In [19]:
#using the best algorithm (have to train again...)

data = pd.read_csv('my_data.csv')

features = ['danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature']

X = data[features]
y = data['Like']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

metrics.accuracy_score(y_test, y_pred)

0.8830188679245283

In [27]:
#passing with the test playlist 

pred = clf.predict_proba(test_df[features])

test_df['prediction'] = pred
test_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.6250,0.03550,0.116,0.502,143.802,216004,4,0.02
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.0443,0.00826,0.142,0.369,125.389,177520,4,0.41
0,6hlJIYn5IBNqcSGTS6X5Xq,My Girls,Animal Collective,0.508,0.953,4,-6.031,0,0.2560,0.04450,0.929,0.555,119.973,340787,4,0.57
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.0577,0.01970,0.343,0.537,117.783,315200,4,0.22
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.1110,0.04490,0.105,0.301,139.389,335333,4,0.44


In [28]:
test_df.describe()

,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
count,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000
mean,0.553425,0.705874,5.070588,-6.986335,0.621849,0.168742,0.156554,0.189262,0.523689,125.530623,236733.375630,3.957143,0.511748
std,0.156678,0.193760,3.634022,2.745104,0.485130,0.247247,0.266504,0.153942,0.234269,27.622676,58971.556673,0.276367,0.299308
min,0.000000,0.040800,0.000000,-25.153000,0.000000,0.000003,0.000000,0.022000,0.000000,0.000000,71912.000000,0.000000,0.000000
25%,0.451000,0.584500,2.000000,-8.311000,0.000000,0.004783,0.000116,0.096725,0.344250,104.632500,201807.000000,4.000000,0.220000
50%,0.563000,0.737000,5.000000,-6.611500,1.000000,0.044400,0.008095,0.125500,0.525000,122.991000,229213.000000,4.000000,0.540000
75%,0.659000,0.865000,8.000000,-5.162750,1.000000,0.225000,0.183250,0.237500,0.712000,141.939500,262686.500000,4.000000,0.777500
max,0.975000,0.998000,11.000000,1.342000,1.000000,0.980000,0.974000,0.961000,0.976000,216.209000,670667.000000,5.000000,1.000000


In [31]:
liked_test_songs = test_df[test_df['prediction'] >0.95]
liked_test_songs

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0mBkoM8r7KAQzZij5swTUL,Walking On A Dream,Empire of the Sun,0.870,0.699,5,-5.541,0,0.245000,0.000007,0.0588,0.726,126.967,198440,4,1.00
0,2r8xik8WxmvWDqj5kbyc63,Bloodvessels,Juba,0.456,0.959,9,-6.192,0,0.003460,0.009450,0.3210,0.542,150.033,239999,4,0.97
0,69x16JmnJxuLVP9ELBoY03,i,Kendrick Lamar,0.750,0.884,1,-5.395,1,0.024600,0.000004,0.2380,0.867,121.915,231933,4,0.96
0,1hjQ3EARnSDznlLaXaWAhf,So Hot You're Hurting My Feelings,Caroline Polachek,0.795,0.576,6,-4.971,1,0.078700,0.000010,0.1210,0.847,111.975,183867,4,0.98
0,7d8GetOsjbxYnlo6Y9e5Kw,Mr. Brightside,The Killers,0.352,0.928,1,-3.710,1,0.001130,0.000000,0.0987,0.239,148.026,222107,4,0.96
0,3HOwzZCW4jgwls1nX215Ie,Then There Were Two,Spies in Disguise,0.657,0.771,0,-4.268,1,0.265000,0.000000,0.1030,0.793,164.749,152001,4,0.97
0,3oqBOAsqRMKUHc8MGx4o72,Death,White Lies,0.454,0.832,7,-5.649,1,0.000451,0.018800,0.0760,0.228,135.087,301213,4,0.98
0,11NbY6pumU3yzLyo7P5oNQ,Tick Tick Boom,The Hives,0.565,0.831,9,-3.102,0,0.000384,0.000012,0.2450,0.498,135.111,205413,4,0.96
0,4UyAtnwhaKv7EG1BdkBYRI,Ready To Let Go,Cage The Elephant,0.719,0.896,1,-3.263,1,0.011000,0.000032,0.0731,0.746,100.962,187973,4,0.97
0,0V2SavjRuimH4TIbrKyyN0,How Deep Is Your Love?,The Rapture,0.472,0.830,9,-3.938,0,0.000650,0.000589,0.2120,0.136,122.316,385920,4,0.99
